<font color='red'>**Note:** non-HBP members should contact “support@humanbrinproject.eu” for access to the validation tools</font>

## About this test
This test shall take as input a BluePyOpt optimized output file. The validation test would then evaluate the model for all parameter sets against various eFEL features. It should be noted that the reference data used is that located within the model, so this test can be considered as a quantification of the goodness of fitting the model. The results are registered on the HBP Validation Framework app. If an instance of the Model Catalog and Validation Framework are not found in the current Collab, then these are created. Additionally, a test report is generated and this can be viewed within the Jupyter notebook, or downloaded.

In [ ]:
import os
import pkg_resources
from pkg_resources import parse_version
!pip install -q tornado==4.5.3

<font color='red'>**Note:** If you encounter any errors in the below cell, please try to restart the kernel (Kernel -> Restart & Run All)</font>

In [ ]:
req_packages = [    
                    {"hbp_service_client"       : {"min_version": "1.1.0",  "install_version": "1.1.0"}},
                    {"sciunit"                  : {"min_version": "0.2.1",  "install_version": "0.2.1"}},
                    {"neuronunit"               : {"min_version": "0.1.8.2","install_version": "0.1.8.2"}},
                    {"eFELunit"                 : {"min_version": "1.1.4",  "install_version": "1.1.4"}},
                    {"bluepyopt"                : {"min_version": "1.6.22", "install_version": "1.6.42"}},                    
                    {"hbp_validation_framework" : {"min_version": "", "install_version": "https://github.com/appukuttan-shailesh/hbp-validation-client@usecase_patching"}},
                    {"numpy"                    : {"min_version": "1.16.2", "install_version": "1.16.2"}},    
                    {"neo"                      : {"min_version": "0.6.1",  "install_version": "0.6.1"}},
                    {"fpdf"                     : {"min_version": "1.7.2",  "install_version": "1.7.2"}},
                    {"PyPDF2"                   : {"min_version": "1.26.0", "install_version": "1.26.0"}},
                    {"tornado"                  : {"min_version": "4.5.3",  "install_version": "4.5.3"}}
                ]

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg in req_packages:        
        for pkg_name, pkg_vinfo in pkg.items():
            print("Checking for package: {}".format(pkg_name))        
            try:
                pkg_resources.get_distribution(pkg_name)        
                current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
                print("\t{}: current version = {}".format(pkg_name, current_version))
                if not pkg_vinfo["min_version"] or current_version < parse_version(pkg_vinfo["min_version"]) or current_version > parse_version(pkg_vinfo["install_version"]):                                                
                        print("\tInstalling another version of {}.".format(pkg_name))
                        raise
            except:            
                if "github.com" in pkg_vinfo["install_version"]:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall git+{}".format(pkg_vinfo["install_version"]))
                else:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                print("\t{}: installed version = {}".format(pkg_name, pkg_vinfo["install_version"]))
                if pkg_name == "hbp_service_client":
                    from IPython.display import HTML
                    display(HTML('''<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
                    Jupyter.notebook.dirty = false; window.location.reload(); })</script>'''))

install_req_packages()                 

In [ ]:
import sciunit
import bluepyopt.ephys as ephys
from eFELunit.utils import CellModel
from hbp_validation_framework import utils, TestLibrary, ModelCatalog

import json
import requests
import urllib2
import StringIO
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from pandas.io.json import json_normalize

### Check if Model Catalog and Validation Framework Apps Exist in Collab
If the notebook is run inside a Collab, we check if an instance of the Model Catalog and Validation Framework apps exist in the current Collab. If not, we add an instance of each (this will be reflected in the Collab's navigation panel, possibly on reloading the page).

NOTE: **HBP_USERNAME** is an optional parameter when the notebook is being run inside the Collaboratory. The notebook can automatically identify your username in this scenario. This parameter needs to be specified if a user wishes to download the notebook and run it locally. Another potential (less likely) reason for specifying this (even within the Collaboratory) is in dealing with access permissions (wanting to run the test with different credentials).

NOTE: Even if this notebook is not run inside a Collab, the following cell needs to be executed. It will identify if environment and manage accordingly. When not run inside a Collab, it will simply setup parameters required for the test, and not attempt to create new apps.

In [ ]:
# your HBP username; not essential if running inside the Collaboratory
HBP_USERNAME = ""
testLibrary = TestLibrary(username=HBP_USERNAME, environment="integration")
modelCatalog = ModelCatalog.from_existing(testLibrary)

try:
    collab_path = get_collab_storage_path()
    collab_id = collab_path[1:] # this might fail for very old Collabs which use name instead of Collab ID
except:
    # not run inside Collaboratory
    print("\nPlease enter a Collab ID where you wish to store the results:")
    print("E.g.: 8123")
    print("Note: you should be a member of this Collab!")
    collab_id = input()
    if not isinstance(collab_id, int):
        raise ValueError("Possibly invalid Collab ID: {}. Numeric input expected!".format(collab_id))    

# check if apps exist; if not then create them
MCapp_navID = modelCatalog.exists_in_collab_else_create(collab_id)
modelCatalog.set_app_config(collab_id=collab_id, app_id=MCapp_navID, only_if_new="True")
VFapp_navID = testLibrary.exists_in_collab_else_create(collab_id)
testLibrary.set_app_config(collab_id=collab_id, app_id=VFapp_navID, only_if_new="True")

### Model Selection: Specifying model from ModelCatalog
Hippocampus models registered on the Model Catalog and known to be in the BluePyOpt format are listed below.

In [ ]:
l1 = modelCatalog.list_models(brain_region="hippocampus", collab_id="12027")#, author="Rosanna Migliore")
l2 = modelCatalog.list_models(brain_region="hippocampus", collab_id="9821")#, author="Rosanna Migliore")
# l3 = modelCatalog.list_models(brain_region="hippocampus", collab_id="12027", author="Paola Vitale")
# l4 = modelCatalog.list_models(brain_region="hippocampus", collab_id="9821", author="Paola Vitale")
list_of_models = []
list_of_models.extend(l1)
list_of_models.extend(l2)
# list_of_models.extend(l3)
# list_of_models.extend(l4)
for item in list_of_models:
    if item["author"][0]["family_name"] not in ["Migliore", "Vitale"]:
        list_of_models.remove(item)
print len(list_of_models) 
df = pd.DataFrame.from_dict(json_normalize(list_of_models), orient='columns')
df = df.reindex(['name', 'id', 'author', 'brain_region', 'species', 'cell_type', 'model_scope', 'abstraction_level', 'description'], axis=1)
df.index += 1
df

In [ ]:
print("\nEnter the # of required model: ")
choice = input()
if choice <= len(list_of_models) and choice > 0:
    model_id = list_of_models[choice-1]["id"]
    model_name = list_of_models[choice-1]["name"]
    model_instances = modelCatalog.list_model_instances(model_id=model_id)    
    model_instance_json = max(model_instances, key=lambda x:x['timestamp'])
    file_path = model_instance_json["source"]
else:
    raise ValueError("Invalid entry for model choice!")

try:        
    response = requests.get(file_path)
    zip_ref = zipfile.ZipFile(StringIO.StringIO(response.content))
    zip_ref.extractall()
    model_path = os.path.join(os.getcwd(),model_name)

    meta_info = requests.get(file_path.replace(".zip", "_meta.json"))    
    if meta_info.status_code == 200:
        with open(os.path.join(model_path, model_name+"_meta.json"), 'w') as f:
            json.dump(meta_info.json(), f)
    
    model_image_url = file_path.replace(model_name+".zip", "_".join(model_name.split("_")[3:-1])+"_morph.jpeg")
    model_image = requests.get(model_image_url)    
    model_image_localPath = None
    if model_image.status_code == 200:        
        model_image_localPath = os.path.join(model_path, model_name.split("_")[3]+"_morph.jpg")
        with open(model_image_localPath, 'wb') as f:
            f.write(model_image.content)         
        print "\nModel Morphology:"
        img = mpimg.imread(model_image_localPath)
        imgplot = plt.imshow(img)
        plt.show()
except:
    raise IOError("Model url = {} is invalid!".format(file_path))

 ### Instantiating the model; Running validation
The usecase will run validations on the specified model. At the end of the test, the user is provided with a textual summary of the score and the path to related output files generated by the test. These and other details can be viewed in the Validation Framework app (see Collab's Navigation panel; select Validation Framework).

In [ ]:
cell_model = CellModel(model_path=model_path, model_name=model_name, run_alerts=True)
cell_model.model_uuid = model_id
cell_model.source = file_path
cell_model.model_version = "1.0"

result_uuid, score = utils.run_test(username=HBP_USERNAME, environment="integration", model=cell_model, test_alias="bpo_efel", test_version="1.0", storage_collab_id=collab_id, register_result=True, client_obj=testLibrary, observation_dir=cell_model.base_path, plot_figure=True)

print("The result(s) can be viewed in the HBP Validation Framework app. Direct link:")
print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))

### Score Summary

In [ ]:
if result_uuid:    
    df, excluded_results = utils.generate_score_matrix(environment="integration", result_list=[result_uuid], collab_id=collab_id, client_obj=modelCatalog)        
    from IPython.core.display import HTML
    HTML("<style>.rendered_html th {max-width: 120px;}</style>")
    display(df)

### Generate Report
The validation framework can generate a PDF report for a successfully completed test. The user is prompted whether such a report should be generated for the current tests. If asked to generate, the location to the generated PDF is indicated.

In [ ]:
report_path = None
if result_uuid:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        valid_uuids, report_path = utils.generate_report(environment="integration", result_list=[result_uuid], only_combined=True, client_obj=modelCatalog)    

### View Report Inside Jupyter Notebook
The PDF report created in the above cell is displayed within the Jupyter notebook. This can also be downloaded by clicking the download button inside the display frame.

In [ ]:
print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))